In [62]:
import yfinance as yf
import pandas as pd

from fastapi.responses import JSONResponse

import numpy as np
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


In [2]:
def read_yfinance():
    nvda = yf.Ticker("NVDA").history(period="1mo").reset_index()
    nvda['Date'] = pd.to_datetime(nvda['Date']).dt.date
    nvda = nvda.drop(["Dividends", "Stock Splits"], axis=1)
    return nvda

In [34]:
read_yfinance()

,Date,Open,High,Low,Close,Volume
0,2024-10-09,134.110001,134.520004,131.380005,132.649994,246191600
1,2024-10-10,131.910004,135.000000,131.000000,134.809998,242311300
2,2024-10-11,134.009995,135.779999,133.660004,134.800003,170209500
3,2024-10-14,136.470001,139.600006,136.300003,138.070007,232347700
4,2024-10-15,137.869995,138.570007,128.740005,131.600006,377831000
5,2024-10-16,133.979996,136.619995,131.580002,135.720001,264879700
6,2024-10-17,139.339996,140.889999,136.869995,136.929993,306435900
7,2024-10-18,138.669998,138.899994,137.279999,138.000000,176090200
8,2024-10-21,138.130005,143.710007,138.000000,143.710007,264554500
9,2024-10-22,142.910004,144.419998,141.779999,143.589996,226311600


In [36]:
# data = read_yfinance().select_dtypes(include=np.number)
data = pd.DataFrame(read_yfinance()['Close'])
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [37]:
data = scaled_data

print(len(data))
print(range(len(data)-60-1))
print(data[1:(1+60), 0])

23
range(0, -38)
[0.18576341 0.18518502 0.37442139 0.         0.23842566 0.30844832
 0.37037004 0.70081027 0.69386522 0.46064769 0.50983786 0.57523078
 0.51620363 0.55844876 0.44791613 0.06712896 0.21990672 0.25752299
 0.48090267 0.81076363 1.         0.92766203]


In [38]:
scaled_data

array([[0.06076319],
       [0.18576341],
       [0.18518502],
       [0.37442139],
       [0.        ],
       [0.23842566],
       [0.30844832],
       [0.37037004],
       [0.70081027],
       [0.69386522],
       [0.46064769],
       [0.50983786],
       [0.57523078],
       [0.51620363],
       [0.55844876],
       [0.44791613],
       [0.06712896],
       [0.21990672],
       [0.25752299],
       [0.48090267],
       [0.81076363],
       [1.        ],
       [0.92766203]])

In [40]:
scaled_data[0:3,0]

array([0.06076319, 0.18576341, 0.18518502])

In [41]:
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data)-time_step-1):
        X.append(data[i:(i+time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

In [43]:
print(len(scaled_data))

23


In [48]:
time_step = 3

X, y = create_dataset(scaled_data, time_step)
print(X)

[[0.06076319 0.18576341 0.18518502]
 [0.18576341 0.18518502 0.37442139]
 [0.18518502 0.37442139 0.        ]
 [0.37442139 0.         0.23842566]
 [0.         0.23842566 0.30844832]
 [0.23842566 0.30844832 0.37037004]
 [0.30844832 0.37037004 0.70081027]
 [0.37037004 0.70081027 0.69386522]
 [0.70081027 0.69386522 0.46064769]
 [0.69386522 0.46064769 0.50983786]
 [0.46064769 0.50983786 0.57523078]
 [0.50983786 0.57523078 0.51620363]
 [0.57523078 0.51620363 0.55844876]
 [0.51620363 0.55844876 0.44791613]
 [0.55844876 0.44791613 0.06712896]
 [0.44791613 0.06712896 0.21990672]
 [0.06712896 0.21990672 0.25752299]
 [0.21990672 0.25752299 0.48090267]
 [0.25752299 0.48090267 0.81076363]]


In [49]:
print(len(X))

19


In [50]:
X.shape[0]

19

In [51]:
X.shape[1]

3

In [53]:
X = X.reshape(X.shape[0], X.shape[1], 1)
print(X)

[[[0.06076319]
  [0.18576341]
  [0.18518502]]

 [[0.18576341]
  [0.18518502]
  [0.37442139]]

 [[0.18518502]
  [0.37442139]
  [0.        ]]

 [[0.37442139]
  [0.        ]
  [0.23842566]]

 [[0.        ]
  [0.23842566]
  [0.30844832]]

 [[0.23842566]
  [0.30844832]
  [0.37037004]]

 [[0.30844832]
  [0.37037004]
  [0.70081027]]

 [[0.37037004]
  [0.70081027]
  [0.69386522]]

 [[0.70081027]
  [0.69386522]
  [0.46064769]]

 [[0.69386522]
  [0.46064769]
  [0.50983786]]

 [[0.46064769]
  [0.50983786]
  [0.57523078]]

 [[0.50983786]
  [0.57523078]
  [0.51620363]]

 [[0.57523078]
  [0.51620363]
  [0.55844876]]

 [[0.51620363]
  [0.55844876]
  [0.44791613]]

 [[0.55844876]
  [0.44791613]
  [0.06712896]]

 [[0.44791613]
  [0.06712896]
  [0.21990672]]

 [[0.06712896]
  [0.21990672]
  [0.25752299]]

 [[0.21990672]
  [0.25752299]
  [0.48090267]]

 [[0.25752299]
  [0.48090267]
  [0.81076363]]]


In [59]:

model = Sequential()
model.add(LSTM(units=50, activation='relu', return_sequences=False, input_shape=(X.shape[1], 1)))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 50)             │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,451 (40.82 KB)

 Trainable params: 10,451 (40.82 KB)

 Non-trainable params: 0 (0.00 B)

In [60]:
model.fit(X, y, epochs=20, batch_size=15)

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.3083
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2650
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2633
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2426 
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2624  
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2512
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2434  
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2181
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2193   
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2135   
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2130 
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1796
Epoch 13/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1830   
Epoch 14/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1799  
Epoch 15/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1681
Epoch 16/20
2/2 ━━━━━━━━━━━━━━━

In [11]:
predicted_price = model.predict(X)
predicted_price = scaler.inverse_transform(predicted_price)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


ValueError: non-broadcastable output operand with shape (17,1) doesn't match the broadcast shape (17,5)

In [10]:
print(predicted_price)

[[0.25703034]
 [0.2661753 ]
 [0.31273273]
 [0.34713826]
 [0.36194795]
 [0.39094648]
 [0.4537224 ]
 [0.4706244 ]
 [0.49057022]
 [0.52282614]
 [0.50801027]
 [0.4922849 ]
 [0.47218457]
 [0.42982727]
 [0.37813434]
 [0.35255167]
 [0.36456817]]


In [57]:
model.save('teste_model.keras')

In [63]:
load_teste_model = load_model('teste_model.keras')

In [67]:
train_predictions = model.predict(23).flatten()
train_result = pd.DataFrame(data={'prediction':train_predictions,'original':y})

ValueError: Unrecognized data type: x=23 (of type <class 'int'>)

In [65]:
train_result

,prediction,original
0,0.155556,0.374421
1,0.167770,0.000000
2,0.158504,0.238426
3,0.161802,0.308448
4,0.160754,0.370370
5,0.174015,0.700810
6,0.193237,0.693865
7,0.209596,0.460648
8,0.211900,0.509838
9,0.203952,0.575231
